# Código Ejercicio 2

In [1]:
!pip install pyopencl
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!lscpu
!nvidia-smi

Arquitectura:                            x86_64
  modo(s) de operación de las CPUs:      32-bit, 64-bit
  Address sizes:                         36 bits physical, 48 bits virtual
  Orden de los bytes:                    Little Endian
CPU(s):                                  2
  Lista de la(s) CPU(s) en línea:        0,1
ID de fabricante:                        GenuineIntel
  Nombre del modelo:                     Intel(R) Core(TM)2 Duo CPU     P8700  @
                                          2.53GHz
    Familia de CPU:                      6
    Modelo:                              23
    Hilo(s) de procesamiento por núcleo: 1
    Núcleo(s) por «socket»:              2
    «Socket(s)»                          1
    Revisión:                            10
    CPU MHz máx.:                        2527,0000
    CPU MHz mín.:                        798,0000
    BogoMIPS:                            5041.87
    Indicadores:                         fpu vme de pse tsc msr pae mce cx8 apic
  

## Kernel

In [3]:
KernelSource = """
__kernel void parallelMMult(
  const int N,
  __global const float *a_g,
  __global const float *b_g,
  __global float *res_g)
  {
    int i = get_global_id(0);
    int j = get_global_id(1);

    float sum = 0.0f;
    for(int k = 0; k < N; k++) {
      sum += a_g[i*N + k] * b_g[k*N + j];
    }
    //barrera
    res_g[i*N + j] = sum;

  }
"""

## Host

In [4]:
N = 2500

# Obtener la plataforma
plataform_list = cl.get_platforms()

# Obtener los dispositivos
devices = plataform_list[0].get_devices(device_type = cl.device_type.GPU)

# Crear el contexto
context = cl.Context(devices=devices)

# Crear el Command Queue
queue = cl.CommandQueue(context)

# Crear e inicializar los vectores de entrada
a_np = np.arange(N*N).astype(np.float32)
b_np = np.arange(N*N).astype(np.float32)

# Crear e inicializar el vector de salida
res_np = np.empty_like(a_np)

# Crear e inicializar los Buffers (OpenCL)
a_g = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf = a_np)
b_g = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf = b_np)

# Crear buffer de salida (OpenCL)
res_g = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, res_np.nbytes)

# Crear el programa
program = cl.Program(context, KernelSource).build()

# Crear el kernel
kernel = program.parallelMMult

# Configurar los argumentos
kernel.set_scalar_arg_dtypes([np.int32, None, None, None])
kernel.set_args(N, a_g, b_g, res_g)

# Definir el espacio indexado
globalRange = (N,N)
localRange = None

# Ejecutar el kernel
ev = cl.enqueue_nd_range_kernel(queue, kernel, globalRange, localRange)

# Copy result del device al host
cl.enqueue_copy(queue, res_np, res_g)

LogicError: Context failed: INVALID_VALUE

In [5]:
np_matmul = np.matmul(a_np.reshape(N,N),b_np.reshape(N,N))

print(res_np.reshape(N,N))

print(np_matmul)

NameError: name 'a_np' is not defined

In [ ]:
print(a_np.reshape(N,N))


[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]
